## How to use the client to query the KG

In [2]:
# wir benötigen wieder ein paar python imports...
import os
import sys
from os import environ

sys.path.append(os.path.abspath(".."))


from client.connector_client import ConnectorClient
from client.sparql_builder import *  # todo: remove later
from client.sparql_builder import SPARQLBuilder

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])
builder = SPARQLBuilder()

### 🧾 `build_get_properties_query()` — Retrieve Properties of an Entity

This method builds a SPARQL query to fetch properties of a given entity (subject) in the knowledge graph. It supports fetching **all properties** or a **specific list of properties**. For doing so, it requires an explicit entity URI such as e.g. a DataPoint URI or else. As these URIs are typically machine-created and thus UUID-based, the build_get_properties_query() is typically executed after first identifying an entity of interest.

---

#### 📥 **Arguments**

| Parameter        | Type         | Description                                                                 |
|------------------|--------------|-----------------------------------------------------------------------------|
| `subject_uri`    | `str`        | URI or prefixed name of the entity (e.g., `df_instance:1234abc`)            |
| `property_uris`  | `list[str]`  | *(Optional)* List of properties to retrieve (e.g., `["df:unit", "df:label"]`). If `None`, all properties will be returned. |

---

#### 📤 **Returns**

A SPARQL query string that:

- Returns **all** triples for the subject (`?p ?o`) if no property list is given.
- Or returns **only** the selected properties as `OPTIONAL` clauses, using one row per subject with selected variables (e.g., `?unit`, `?label`, etc.).

---

#### ✅ **Example Usage**

In [3]:
selected_datapoint = "d5e80e76-1048-4c42-ac1b-188abb3d50ff" # Connector
selected_datapoint_uri = f"http://stephantrattnig.org/instances#{selected_datapoint}" # instance URI
output = await client.get_properties(selected_datapoint_uri)
print(output)

[]


### 📋 `build_list_instances_query()` — List Instances of a Class

This method builds a SPARQL query to retrieve all instances of a given RDF class in the knowledge graph. Optionally, it can include selected properties for each instance.

---

#### 📥 **Arguments**

| Parameter         | Type         | Description                                                                 |
|-------------------|--------------|-----------------------------------------------------------------------------|
| `class_uri`       | `str`        | URI or prefixed name of the RDF class (e.g., `df:DataPoint`)               |
| `optional_props`  | `list[str]`  | *(Optional)* List of property URIs to include in the results (e.g., `["df:label", "df:unit"]`) |

---

#### 📤 **Returns**

A SPARQL query string that:

- Lists all instances (`?instance`) of the specified class.
- Optionally includes properties as separate variables (e.g., `?label`, `?unit`) via `OPTIONAL` clauses.
- Returns one row per instance, enriched with additional metadata if available.

---

#### ✅ **Example Usage**

In [9]:
output = await client.list_instances(
    class_uri="df:Connector",
    optional_props=[
        "rdfs:label",
        "df:ConnectorAddress",
        "df:moduleId",
        "df:moduleType"
    ]
)
print(output)

                               instance                           label  \
0  d5e80e76-1048-4c42-ac1b-188abb3d50ff  Connector: opcua_dev_connector   
1  748302f0-7559-47f9-a799-0930c681b259  Connector: opcua_dev_connector   
2  8a263c62-e897-4202-aa26-62ed5fc8b84c  Connector: opcua_dev_connector   

  ConnectorAddress             moduleId          moduleType  
0    10.157.101.85  opcua_dev_connector  ServiceModuleOPCUA  
1    10.157.101.85  opcua_dev_connector  ServiceModuleOPCUA  
2    10.157.101.85  opcua_dev_connector  ServiceModuleOPCUA  


In [12]:
output = await client.list_instances(
    class_uri="df:DataPoint", 
    optional_props=["df:hasDataType", "rdfs:label"]
)
output

,instance,hasDataType,label
0,35af9f26-4818-458b-9a44-2f15cd140b88,float,DataPoint: AlignmentOffset
1,12281921-0815-4373-ba4e-1743bf4d4f77,float,DataPoint: ClampingForce
2,f87e73ae-f4c4-4ec4-91df-23776a6e40c2,float,DataPoint: ScrewdriverTorque
3,33573786-e874-4508-b0b3-e2a81741d92d,float,DataPoint: CycleTime
4,8a335dee-5c7c-4d63-9391-35ae253ab998,float,DataPoint: MaxClampingForce
5,a346b5e3-e7a8-428e-b663-c42be2630ff9,float,DataPoint: TorqueThreshold
6,026d047a-e6de-4ef8-ad7e-8ce915b048a5,float,DataPoint: AlignmentOffset
7,1cd10425-c663-4b5a-ac36-5632d35c45c6,float,DataPoint: ClampingForce
8,b1c708d6-a010-4099-aa17-a44a92379b16,float,DataPoint: ScrewdriverTorque
9,0cb14aed-4ad3-4ac6-9ff1-e2026eda590f,float,DataPoint: CycleTime


### 🔍 `search_entity_query()` — Entity Search by Keyword

This method constructs a SPARQL query to search for entities in the knowledge graph using a keyword. It supports class-based filtering, property-based matching, and flexible match modes.

---

#### 📥 **Arguments**

| Parameter      | Type     | Description                                                                 |
|----------------|----------|-----------------------------------------------------------------------------|
| `keyword`      | `str`    | The keyword to search for (e.g., `"Torque"`)                                |
| `class_uri`    | `str`    | *(Optional)* Restrict search to a specific RDF class (e.g., `df:DataPoint`) |
| `property_uri` | `str`    | RDF property to match the keyword against (default: `rdfs:label`)           |
| `match_mode`   | `str`    | `"fuzzy"` for substring matching (default), `"exact"` for full match        |

---

#### 📤 **Returns**

A SPARQL query string that retrieves up to 50 matching entities with:

- Their URI (`?instance`)
- Their `rdfs:label` if available (`?label`)

---

#### ✅ **Example Usage**

In [16]:
output = await client.search_entity(
    keyword="Torque",
    class_uri="df:DataPoint",
    property_uri="rdfs:label",
    match_mode="fuzzy"
)
output

,instance,label
0,f87e73ae-f4c4-4ec4-91df-23776a6e40c2,DataPoint: ScrewdriverTorque
1,a346b5e3-e7a8-428e-b663-c42be2630ff9,DataPoint: TorqueThreshold
2,b1c708d6-a010-4099-aa17-a44a92379b16,DataPoint: ScrewdriverTorque
3,3c3cc7f7-f136-41d7-a802-3403a309be5d,DataPoint: TorqueThreshold
4,da229473-2480-4ab0-8fed-de978f9cde3f,DataPoint: ScrewdriverTorque
5,dafac746-25b6-446f-8753-42a3323d2635,DataPoint: TorqueThreshold


In [20]:
output = await client.search_entity(
    keyword="ClampingForce", 
    class_uri="df:DataPoint"
)
output

,instance,label
0,12281921-0815-4373-ba4e-1743bf4d4f77,DataPoint: ClampingForce
1,8a335dee-5c7c-4d63-9391-35ae253ab998,DataPoint: MaxClampingForce
2,1cd10425-c663-4b5a-ac36-5632d35c45c6,DataPoint: ClampingForce
3,3cb906ed-0483-4abd-836d-39ba81f2717a,DataPoint: MaxClampingForce
4,e680a0a8-3ba3-4416-ade3-f39243eb5fcc,DataPoint: ClampingForce
5,da8dbd9e-d048-48e0-aa9b-26641c012449,DataPoint: MaxClampingForce


### 🔗 `build_get_related_query()` — Get Related Objects

This method builds a SPARQL query to retrieve objects linked to a given subject through a specified predicate (i.e., outgoing relationship).

---

#### 📥 **Arguments**

| Parameter        | Type   | Description                                                                 |
|------------------|--------|-----------------------------------------------------------------------------|
| `subject_uri`    | `str`  | URI or prefixed name of the subject entity (e.g., `df_instance:Device_001`) |
| `predicate_uri`  | `str`  | URI or prefixed name of the predicate (e.g., `df:hasDatapoint`)             |

---

#### 📤 **Returns**

A SPARQL query string that:

- Retrieves all related objects for a given subject via the specified predicate.
- Optionally includes the `rdfs:label` of the object for human-friendly display.
- Each result includes:
  - `?object`: the URI of the related entity
  - `?label`: a readable label, if available

---

#### ✅ **Example Usage**

In [22]:
# Get the Topics related to the MessageBroker

subject_uri = "df:MessageBrokerInstance"

output = await client.get_related(
    subject_uri=subject_uri,
    predicate_uri="df:hasTopic"
)
output

,object,label
0,request.ServiceModuleOPCUA.opcua_dev_connector,request.ServiceModuleOPCUA.opcua_dev_connector
1,response.ServiceModuleOPCUA.opcua_dev_connector,request.ServiceModuleOPCUA.opcua_dev_connector
2,telemetry.ServiceModuleOPCUA.opcua_dev_connector,telemetry.ServiceModuleOPCUA.opcua_dev_connector


In [30]:
# first: identify connector instances
output = await client.list_instances(
    class_uri="df:Service"
)
output

,instance
0,ce8ab652-ffc1-4727-9cf8-9b7d1960d02a
1,becde037-32e9-4fa2-b094-7ae464a73c88
2,2c5c78ee-3e6c-427d-98b1-5b78a3817626


In [31]:
# second: get the datapoints relating to these subscriptions:

selected_connector = output.iloc[0]["instance"]
selected_datapoint_uri = f"http://stephantrattnig.org/instances#{selected_connector}" # instance URI
output = await client.get_related(
    subject_uri=selected_datapoint_uri,
    predicate_uri="df:providesDataPoint"
)
output

,object,label
0,35af9f26-4818-458b-9a44-2f15cd140b88,DataPoint: AlignmentOffset
1,12281921-0815-4373-ba4e-1743bf4d4f77,DataPoint: ClampingForce
2,f87e73ae-f4c4-4ec4-91df-23776a6e40c2,DataPoint: ScrewdriverTorque
3,33573786-e874-4508-b0b3-e2a81741d92d,DataPoint: CycleTime
4,8a335dee-5c7c-4d63-9391-35ae253ab998,DataPoint: MaxClampingForce
5,a346b5e3-e7a8-428e-b663-c42be2630ff9,DataPoint: TorqueThreshold


### 🔁 `build_get_related_inverse_query()` — Get Inverse Relations

This method builds a SPARQL query to retrieve all **subjects** that point to a given **object** using a specified predicate (i.e., incoming relationship).

---

#### 📥 **Arguments**

| Parameter         | Type           | Description                                                                 |
|------------------|----------------|-----------------------------------------------------------------------------|
| `object_uri`      | `str`          | URI or prefixed name of the object (e.g., `df_instance:DataPoint_001`)     |
| `predicate_uri`   | `str`          | URI or prefixed name of the predicate (e.g., `df:hasDatapoint`)            |
| `optional_props`  | `list[str]`    | Additional property URIs to return for each subject (default: `None`)      |

---

#### 📤 **Returns**

A SPARQL query string that:

- Finds all subjects that refer to the given object via the specified predicate.
- Includes the `rdfs:label` of each subject by default.
- Optionally includes user-defined properties for each subject.

The result will contain:
- `?subject`: the URI of the referring subject
- `?label`: a human-readable label of the subject (if available)
- Additional optional variables, if provided

---



#### ✅ **Example Usage**

In [33]:
# first: identify Subscription instances
output = await client.list_instances(
    class_uri="df:Subscription"
)
output

,instance
0,4ca06e6b-7472-4dc7-8746-e9569111b8fa
1,a457311f-7448-436f-8516-997df53be59b
2,19fcd73c-d5f9-49be-9f7d-82520021d418
3,027ed788-0464-4da4-9ca1-d557ee165d58
4,09dbcc92-0943-4e6e-ba17-5b1c337f09f7
5,b477bef8-b71a-425d-843f-cc0d8ad8d099
6,cf61dbe8-57f5-446d-be56-6321cfae9640
7,e7119ab0-eb42-46d8-a519-06f5a62ed9a2
8,ce2a9fa5-56c3-4624-a0b8-432e1e0416a8
9,8dd6f8e0-eb41-4000-be6b-97b4558307f0


In [34]:
# based on the subscription instance, find the connector service that is tracking the subscription
object_uri = output.loc[4]["instance"]
object_uri_full = f"http://stephantrattnig.org/instances#{object_uri}"

output = await client.get_related_inverse(
    object_uri=object_uri_full,
    predicate_uri="df:tracksSubscription",
    optional_props=["df:ConnectorAddress", "df:hasStatus"]
)
output

,subject,label,ConnectorAddress
0,748302f0-7559-47f9-a799-0930c681b259,Connector: opcua_dev_connector,10.157.101.85
